In [1]:
import pandas as pd
import numpy as np


### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_c\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |